In [3]:
import pandas as pd
import plotly.express as px

## Pasar de categorías a números para factorial

In [59]:
est = pd.read_excel('../../../data/raw/Est_Con_Pre_110522.xlsx')
print(est.shape)
est.columns

(9500, 102)


Index(['Timestamp', 'Número de lista', 'Edad', 'Sexo', 'Sector vivienda',
       'Internet', 'Uso del dispositivo móvil', 'Nivel escolaridad madre',
       'Nivel escolaridad padre', 'Ocupación madre',
       ...
       'Tipo de discapacidad',
       '¿Te reconoces como una persona con algún tipo de discapacidad?',
       '1.8 Desarrollo de software', 'Conoce GreenTIC', 'Código IE', 'Grupo',
       'eliminar - ¿Qué opinas de cada una de estas carreras? [Fila 8]',
       'eliminar - ¿Qué opinas de cada una de estas carreras? [Fila 7]',
       'eliminar - ¿Qué opinas de cada una de estas carreras? [0,/]',
       'eliminar - ¿Qué opinas de cada una de estas carreras? [Fila 3]'],
      dtype='object', length=102)

In [60]:
est['Timestamp'] = pd.to_datetime(est['Timestamp'])
est = est[est.Timestamp>'2022-04-14']
est['Fecha'] = est.Timestamp.dt.strftime('%d/%m')
est['Fecha'][:5]

0    19/04
1    19/04
2    19/04
3    19/04
4    19/04
Name: Fecha, dtype: object

In [61]:
est = est.drop(columns=est.filter(regex=r'eliminar').columns)
est = est.drop(columns='Timestamp')
est = est.dropna(subset='Número de lista')
est = est[est['Código IE']<300]
est = est.reset_index()
est.shape

(9254, 71)

In [69]:
respuestas_correctas = {
    "Un algoritmo es:":"Una secuencia lógica de pasos para realizar una tarea",
    "¿Para qué sirven los algoritmos?":"Para planificar la solución a un problema",
    "Un bucle es:":"Un conjunto de instrucciones que se ejecuta mientras se cumpla una condición",
    "¿Cuál de las siguientes opciones sí le permite al robot completar la misión de fotografiar cada tortuga?":"a.",
    "¿Qué mensaje deseaba enviar la líder Wayuú?":"c. Nublado",
    "¿Cuál de los siguientes códigos permite que el robot complete su misión sembrando café?":"a.",
    "¿Cuál será la foto con más vistas?":"c) Julio",
    "Ayuda al robot verde a salir del laberinto":"b.",
    "Óscar lleva 2 loncheras a la escuela todos los días ¿Cuál de las siguientes afirmaciones es falsa?":"c) Si Óscar empaca Deditos para merendar, puede hacer Arroz de pollo para almorzar",
    "¿Cuál de las siguientes hamburguesas tiene los ingredientes A, E y F?":"a.",
    "¿Qué botella debe cambiarse de color para que el resultado final sea una botella de color blanco?":"a) La botella B debe ser verde"}

de_acuerdo = {"Totalmente en desacuerdo":1,"En desacuerdo":2, "Neutro":3,
"De acuerdo":4,"Totalmente de acuerdo":5}

interesa_carreras = {"No la conozco":0,"La evitaría":1,"Me interesa poco":2,"Está entre mis preferidas":3}

estereotipo_h = {'a) Seguramente un hombre':2, 'b) Quizás un hombre':1,
                 'c) Creo que puede ser un hombre o una mujer':0,
                 'd) Quizás una mujer': -1, 'e) Seguramente una mujer':-2}

estereotipo_m = {'a) Seguramente un hombre':-2, 'b) Quizás un hombre':-1,
                 'c) Creo que puede ser un hombre o una mujer':0,
                 'd) Quizás una mujer': 1, 'e) Seguramente una mujer':2}
#The scores range from -2= ‘counter- stereotypic answer’ to 2= ‘stereotype congruent answer’ for all items.
genero_estereotipos = {'5.1 ¿Quién crees que ganará el concurso de matemáticas?':estereotipo_h,
                       '5.2 ¿Quién crees que es capitán del barco?':estereotipo_h,
                       '5.3 ¿Quién es la persona excluida de la construcción de la casa de madera?':estereotipo_m,
                       '5.4 ¿Quién crees que es el personaje que está sentado y esperando junto a la ventana?':estereotipo_m,
                       '5.5 ¿Quién es la persona que trabaja en educación?':estereotipo_m,
                       '5.6 ¿Quién crees que es la persona que prefiere estudiar ingeniería?':estereotipo_h}


In [70]:
est.columns.tolist()

['index',
 'Número de lista',
 'Edad',
 'Sexo',
 'Sector vivienda',
 'Internet',
 'Uso del dispositivo móvil',
 'Nivel escolaridad madre',
 'Nivel escolaridad padre',
 'Ocupación madre',
 'Ocupación padre',
 '¿Con quién vives?',
 'Grado',
 '1.1. Ingeniería',
 '2.1 Soy capaz de sacar buenas notas en esta asignatura',
 '2.2 Si me va bien en esta asignatura, me ayudará en mi futura ocupación',
 '2.3 A mis padres les gustaría que eligiera un futuro profesional relacionado a esta asignatura',
 '2.4 Sé de alguien en mi familia que utiliza conocimientos relacionados a esta asignatura en su ocupación',
 'Comentarios 1-2',
 'Un algoritmo es:',
 '¿Para qué sirven los algoritmos?',
 'Un bucle es:',
 '3.1 Siento que soy capaz de explicar lo que es el pensamiento computacional',
 '3.2 Siento que puedo enumerar las sub-habilidades que componen el pensamiento computacional',
 '3.3 Siento que soy capaz de dar ejemplos para explicar las sub-habilidades del pensamiento computacional',
 '3.4 Siento que p

In [64]:
est.filter(regex=r'^4.', axis=1).columns

Index(['4.1 Es alarmante que el ritmo de desaparición de especies en la Amazonia Colombiana sea cada vez mayor.',
       '4.2 El aumento de la temperatura atmosférica se debe al uso creciente y continuado de combustibles fósiles (carbón, petróleo…).',
       '4.3 La acumulación de basura procedente de las ciudades es un problema realmente grave.',
       '4.4 Hay una disminución de la superficie forestal y de áreas naturales en el país.',
       '4.5 El planeta está tan contaminado por productos químicos que ya supone un problema para la salud.',
       '4.6 Conozco los riesgos que representa para la vida humana la desaparición de especies animales y vegetales.',
       '4.7 Me preocupa lo que sucede con la tala de árboles.'],
      dtype='object')

In [93]:
col_index = ['Edad', 'Sexo', 'Código IE', '¿Te reconoces como una persona con algún tipo de discapacidad?']
col_caract = ['index','Edad', 'Sexo', 'Sector vivienda', 'Internet', 'Uso del dispositivo móvil',
              'Nivel escolaridad madre', 'Nivel escolaridad padre', 'Ocupación madre',
              'Ocupación padre', '¿Con quién vives?', 'Grado', 'Código IE',
              'Grupo', 'Conoce GreenTIC', 'Número de lista', '¿Te reconoces como una persona con algún tipo de discapacidad?']
#col_conocePC = est.filter(regex=r'^3.*', axis=1).columns.tolist()[0]
#col_PC = est.filter(regex=r'^3.*', axis=1).columns.tolist()[1:5]
col_autoeficacia = est.filter(regex=r'^3.*', axis=1).columns.tolist()
col_carreras = est.filter(regex=r'^1.', axis=1).columns.tolist()
col_interes = est.filter(regex=r'^2.1|2.2', axis=1).columns.tolist()
col_conocimiento = list(respuestas_correctas.keys())
col_ambiental = est.filter(regex=r'^4.', axis=1).columns.tolist()
col_genero = est.filter(regex=r'^5.', axis=1).columns.tolist()
col_genero

['5.1 ¿Quién crees que ganará el concurso de matemáticas?',
 '5.2 ¿Quién crees que es capitán del barco?',
 '5.3 ¿Quién es la persona excluida de la construcción de la casa de madera?',
 '5.4 ¿Quién crees que es el personaje que está sentado y esperando junto a la ventana?',
 '5.5 ¿Quién es la persona que trabaja en educación?',
 '5.6 ¿Quién crees que es la persona que prefiere estudiar ingeniería?']

In [94]:

df_conocimientos = est[['index']+col_conocimiento].melt(id_vars='index', value_name='Respuesta estudiante', var_name='Pregunta')
df_conocimientos['Respuesta correcta'] = df_conocimientos['Pregunta'].replace(respuestas_correctas)
df_conocimientos['Puntaje conocimiento'] = 1*(df_conocimientos['Respuesta estudiante'] == df_conocimientos['Respuesta correcta'])
df_puntaje = (df_conocimientos.groupby('index')['Puntaje conocimiento'].sum()*100/11).reset_index()
df_puntaje.describe()

,index,Puntaje conocimiento
count,9254.000000,9254.000000
mean,4764.779123,32.538264
std,2745.684477,19.742658
min,1.000000,0.000000
25%,2379.250000,18.181818
50%,4757.500000,27.272727
75%,7144.750000,45.454545
max,9499.000000,100.000000


In [95]:
df_estudiantes = df_conocimientos.pivot_table(index='index', columns='Pregunta', values='Puntaje conocimiento').reset_index()
df_estudiantes = pd.merge(est[col_caract], df_estudiantes, on='index')
df_estudiantes

,index,Edad,Sexo,Sector vivienda,Internet,Uso del dispositivo móvil,Nivel escolaridad madre,Nivel escolaridad padre,Ocupación madre,Ocupación padre,...,Un algoritmo es:,Un bucle es:,"¿Cuál de las siguientes hamburguesas tiene los ingredientes A, E y F?",¿Cuál de las siguientes opciones sí le permite al robot completar la misión de fotografiar cada tortuga?,¿Cuál de los siguientes códigos permite que el robot complete su misión sembrando café?,¿Cuál será la foto con más vistas?,¿Para qué sirven los algoritmos?,¿Qué botella debe cambiarse de color para que el resultado final sea una botella de color blanco?,¿Qué mensaje deseaba enviar la líder Wayuú?,Óscar lleva 2 loncheras a la escuela todos los días ¿Cuál de las siguientes afirmaciones es falsa?
0,1,14.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,Titulo universitario,Trabaja,Trabaja,...,0,0,1,1,1,0,0,1,0,1
1,2,16.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Trabaja,Trabaja,...,0,1,0,0,1,0,0,0,0,0
2,3,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,1,0,1,1,0,0,1,0,0,0
3,4,15.0,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Trabaja,Trabaja,...,1,0,0,0,0,0,0,0,1,0
4,5,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Titulo universitario,Secundaría,Trabaja,Trabaja,...,1,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9249,9495,12.0,Masculino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Cuida el hogar,Cuida el hogar,...,0,1,1,0,0,0,0,0,0,1
9250,9496,16.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Cuida el hogar,Trabaja,...,0,0,1,1,0,0,0,0,1,0
9251,9497,14.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Secundaría,Titulo universitario,Trabaja,Trabaja,...,1,0,1,1,1,1,1,0,1,1
9252,9498,14.0,Femenino,Rural,Sí,Uso propio (solo lo usas tú),Primaria,Secundaría,Trabaja,Trabaja,...,0,0,0,0,0,0,0,0,0,0


In [96]:
carreras_df = est[['index'] + col_carreras].replace(interesa_carreras)
df_estudiantes = pd.merge(df_estudiantes, carreras_df, on='index')
df_estudiantes

,index,Edad,Sexo,Sector vivienda,Internet,Uso del dispositivo móvil,Nivel escolaridad madre,Nivel escolaridad padre,Ocupación madre,Ocupación padre,...,¿Qué mensaje deseaba enviar la líder Wayuú?,Óscar lleva 2 loncheras a la escuela todos los días ¿Cuál de las siguientes afirmaciones es falsa?,1.1. Ingeniería,1.2 Matemáticas,1.3 Educación,1.4 Medicina,1.5 Diseño gráfico,1.6 Química,1.7 Enfermería,1.8 Desarrollo de software
0,1,14.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,Titulo universitario,Trabaja,Trabaja,...,0,1,2,1,1,1,1,2,1,2
1,2,16.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Trabaja,Trabaja,...,0,0,2,1,1,3,3,3,2,3
2,3,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,0,0,2,2,2,2,2,2,2,2
3,4,15.0,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Trabaja,Trabaja,...,1,0,2,0,2,1,1,1,1,3
4,5,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Titulo universitario,Secundaría,Trabaja,Trabaja,...,1,0,1,2,3,3,1,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9249,9495,12.0,Masculino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Cuida el hogar,Cuida el hogar,...,0,1,2,3,2,3,3,3,0,1
9250,9496,16.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Cuida el hogar,Trabaja,...,1,0,3,3,2,2,1,2,2,1
9251,9497,14.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Secundaría,Titulo universitario,Trabaja,Trabaja,...,1,1,3,1,1,3,2,1,2,0
9252,9498,14.0,Femenino,Rural,Sí,Uso propio (solo lo usas tú),Primaria,Secundaría,Trabaja,Trabaja,...,0,0,2,2,3,2,2,2,2,0


In [97]:
genero_df = est[['index']+col_genero].replace(genero_estereotipos)
df_estudiantes = pd.merge(df_estudiantes, genero_df, on='index')
df_estudiantes

,index,Edad,Sexo,Sector vivienda,Internet,Uso del dispositivo móvil,Nivel escolaridad madre,Nivel escolaridad padre,Ocupación madre,Ocupación padre,...,1.5 Diseño gráfico,1.6 Química,1.7 Enfermería,1.8 Desarrollo de software,5.1 ¿Quién crees que ganará el concurso de matemáticas?,5.2 ¿Quién crees que es capitán del barco?,5.3 ¿Quién es la persona excluida de la construcción de la casa de madera?,5.4 ¿Quién crees que es el personaje que está sentado y esperando junto a la ventana?,5.5 ¿Quién es la persona que trabaja en educación?,5.6 ¿Quién crees que es la persona que prefiere estudiar ingeniería?
0,1,14.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,Titulo universitario,Trabaja,Trabaja,...,1,2,1,2,0,0,0,1,0,0
1,2,16.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Trabaja,Trabaja,...,3,3,2,3,0,0,0,0,0,0
2,3,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,2,2,2,2,-1,1,1,0,-1,0
3,4,15.0,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Trabaja,Trabaja,...,1,1,1,3,1,0,1,2,0,-1
4,5,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Titulo universitario,Secundaría,Trabaja,Trabaja,...,1,1,3,1,0,0,-1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9249,9495,12.0,Masculino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Cuida el hogar,Cuida el hogar,...,3,3,0,1,2,0,-2,0,2,2
9250,9496,16.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Cuida el hogar,Trabaja,...,1,2,2,1,-2,2,0,-2,2,-1
9251,9497,14.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Secundaría,Titulo universitario,Trabaja,Trabaja,...,2,1,2,0,0,0,0,0,0,-2
9252,9498,14.0,Femenino,Rural,Sí,Uso propio (solo lo usas tú),Primaria,Secundaría,Trabaja,Trabaja,...,2,2,2,0,0,0,-2,0,1,1


In [98]:
df_ambiente = est[['index']+col_ambiental].replace(de_acuerdo)
df_estudiantes = pd.merge(df_estudiantes, df_ambiente, on='index')
df_estudiantes

,index,Edad,Sexo,Sector vivienda,Internet,Uso del dispositivo móvil,Nivel escolaridad madre,Nivel escolaridad padre,Ocupación madre,Ocupación padre,...,5.4 ¿Quién crees que es el personaje que está sentado y esperando junto a la ventana?,5.5 ¿Quién es la persona que trabaja en educación?,5.6 ¿Quién crees que es la persona que prefiere estudiar ingeniería?,4.1 Es alarmante que el ritmo de desaparición de especies en la Amazonia Colombiana sea cada vez mayor.,"4.2 El aumento de la temperatura atmosférica se debe al uso creciente y continuado de combustibles fósiles (carbón, petróleo…).",4.3 La acumulación de basura procedente de las ciudades es un problema realmente grave.,4.4 Hay una disminución de la superficie forestal y de áreas naturales en el país.,4.5 El planeta está tan contaminado por productos químicos que ya supone un problema para la salud.,4.6 Conozco los riesgos que representa para la vida humana la desaparición de especies animales y vegetales.,4.7 Me preocupa lo que sucede con la tala de árboles.
0,1,14.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,Titulo universitario,Trabaja,Trabaja,...,1,0,0,3,3,4,3,3,4,3
1,2,16.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Trabaja,Trabaja,...,0,0,0,4,4,4,4,4,4,4
2,3,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,0,-1,0,4,3,5,2,5,5,5
3,4,15.0,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Trabaja,Trabaja,...,2,0,-1,5,5,5,5,4,4,5
4,5,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Titulo universitario,Secundaría,Trabaja,Trabaja,...,1,0,0,4,4,4,4,3,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9249,9495,12.0,Masculino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Cuida el hogar,Cuida el hogar,...,0,2,2,5,5,5,5,3,5,5
9250,9496,16.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Cuida el hogar,Trabaja,...,-2,2,-1,1,5,1,5,5,5,5
9251,9497,14.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Secundaría,Titulo universitario,Trabaja,Trabaja,...,0,0,-2,5,4,5,4,4,4,5
9252,9498,14.0,Femenino,Rural,Sí,Uso propio (solo lo usas tú),Primaria,Secundaría,Trabaja,Trabaja,...,0,1,1,3,4,4,4,4,3,4


In [99]:
df_estudiantes = pd.merge(df_estudiantes, est[['index']+col_autoeficacia].astype(int), on='index')
df_estudiantes

,index,Edad,Sexo,Sector vivienda,Internet,Uso del dispositivo móvil,Nivel escolaridad madre,Nivel escolaridad padre,Ocupación madre,Ocupación padre,...,3.1 Siento que soy capaz de explicar lo que es el pensamiento computacional,3.2 Siento que puedo enumerar las sub-habilidades que componen el pensamiento computacional,3.3 Siento que soy capaz de dar ejemplos para explicar las sub-habilidades del pensamiento computacional,3.4 Siento que puedo explicar la forma en que las sub-habilidades del pensamiento computacional se correlacionan con la programación,3.5 Siento que puedo analizar un ejercicio y determinar qué sub-habilidades de pensamiento computacional busca desarrollar,3.6 Siento que puedo resolver problemas a través de programación,3.7 Siento que puedo implementar algoritmos,3.8 Siento que puedo crear un programa de computador,3.9 Siento que puedo automatizar tareas a través de la programación,3.10 Siento que puedo utilizar la computación para resolver problemas simples
0,1,14.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,Titulo universitario,Trabaja,Trabaja,...,3,1,2,2,3,3,2,1,2,3
1,2,16.0,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Trabaja,Trabaja,...,5,4,5,5,5,5,5,5,5,5
2,3,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,1,1,2,1,1,1,3,2,2,2
3,4,15.0,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Trabaja,Trabaja,...,2,3,2,3,4,4,4,1,3,5
4,5,14.0,Femenino,Urbana,Sí,Uso compartido con otras personas,Titulo universitario,Secundaría,Trabaja,Trabaja,...,3,3,3,3,3,3,4,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9249,9495,12.0,Masculino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Cuida el hogar,Cuida el hogar,...,3,1,4,3,5,5,3,5,5,5
9250,9496,16.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Cuida el hogar,Trabaja,...,5,4,4,5,5,4,4,5,5,5
9251,9497,14.0,Femenino,Urbana,Sí,Uso propio (solo lo usas tú),Secundaría,Titulo universitario,Trabaja,Trabaja,...,4,3,2,2,2,3,4,3,4,4
9252,9498,14.0,Femenino,Rural,Sí,Uso propio (solo lo usas tú),Primaria,Secundaría,Trabaja,Trabaja,...,1,1,1,1,1,1,1,1,3,4


In [100]:
df_estudiantes.to_excel('Estudiantes_numerico_110522.xlsx')

In [101]:
set(est.columns).difference(set(df_estudiantes.columns))

{'2.1 Soy capaz de sacar buenas notas en esta asignatura',
 '2.2 Si me va bien en esta asignatura, me ayudará en mi futura ocupación',
 '2.3 A mis padres les gustaría que eligiera un futuro profesional relacionado a esta asignatura',
 '2.4 Sé de alguien en mi familia que utiliza conocimientos relacionados a esta asignatura en su ocupación',
 'Comentarios 1-2',
 'Comentarios P3',
 'Comentarios conocimiento',
 'Comentarios género',
 'Comentarios medioambiente',
 'Deseo participar en el estudio',
 'Fecha',
 'Tipo de discapacidad'}